In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
df_train=pd.read_csv('./ag-news-classification-dataset/train.csv')
df_train.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [4]:


df_train['News'] = df_train['Title']+" "+df_train['Description']



In [5]:


df_train.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Class Index  120000 non-null  int64 
 1   Title        120000 non-null  object
 2   Description  120000 non-null  object
 3   News         120000 non-null  object
dtypes: int64(1), object(3)
memory usage: 3.7+ MB


In [6]:


df_train.shape



(120000, 4)

In [7]:


df_train['Class Index'].value_counts()



Class Index
3    30000
4    30000
2    30000
1    30000
Name: count, dtype: int64

In [9]:
!pip install nltk

import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer

from nltk import pos_tag
import matplotlib.pyplot as plt

     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.0 kB 330.3 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.0 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 291.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.5 MB 991.0 kB/s eta 0:00:02
   - -------------------------------------- 0.1/1.5 MB 1.7 MB/s eta 0:00:01
   -- ------------------------------------- 0.1/1.5 MB 744.7 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.5 MB 774.0 kB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.5 MB 807.1 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.5 MB 808.4 kB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.5 MB 838.1 kB/s eta 0:00:02
  

In [16]:
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Ensure you download the necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')

# Define the clean function
def clean(text):
    stem = PorterStemmer()
    text = text.lower()
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    text = text.split()
    text = [i for i in text if i not in stopwords.words('english')]
    text = [stem.stem(i) for i in text]
    return ' '.join(text)

# Apply the clean function to the 'News' column
df_train['clean'] = df_train['News'].apply(clean)
print(df_train['clean'].head())




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bawantha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bawantha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


0    wall st bear claw back black reuter reuter sho...
1    carlyl look toward commerci aerospac reuter re...
2    oil economi cloud stock outlook reuter reuter ...
3    iraq halt oil export main southern pipelin reu...
4    oil price soar time record pose new menac us e...
Name: clean, dtype: object


In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [18]:
sent_len = df_train['clean'].map(lambda x: len(x.split())).max()
sent_len

142

In [19]:
voc_size = 10000
onehot_repr_train=[one_hot(words,voc_size)for words in df_train['clean']] 

In [25]:
embedded_docs_xtrain=pad_sequences(onehot_repr_train,padding='post',maxlen=sent_len)
print(embedded_docs_xtrain)

[[1686 2750 1288 ...    0    0    0]
 [4443 5263 5079 ...    0    0    0]
 [7581 1707 9998 ...    0    0    0]
 ...
 [4563  792  467 ...    0    0    0]
 [1984  181 5482 ...    0    0    0]
 [9097 3307 8657 ...    0    0    0]]


In [27]:
from keras.models import Sequential
from keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Dropout

# Define voc_size and sent_len with appropriate values
voc_size = 5000  # Example vocabulary size
sent_len = 100   # Example sentence length
embedding_vector_features = 40

# Build the model
model = Sequential()
model.add(Input(shape=(sent_len,)))
model.add(Embedding(input_dim=voc_size, output_dim=embedding_vector_features, input_length=sent_len))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))

# Display the model summary
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ (None, 100, 40)             │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_5 (Bidirectional)      │ (None, 256)                 │         173,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 4)                   │             260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 480,260 (1.83 MB)

 Trainable params: 480,260 (1.83 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:


y = df_train['Class Index'].apply(lambda x: x-1)



In [29]:
y = pd.get_dummies(y,dtype=int)

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(embedded_docs_xtrain, y, test_size=0.3, random_state=42)

In [31]:
cb = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",verbose=1)

In [33]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
# Train the model
ag = model.fit(X, y_train, validation_data=(X, y_test), batch_size=256, epochs=20)

NameError: name 'X' is not defined